In [70]:
!pip install PyPDF2 langchain langchain-huggingface faiss-cpu langchain-groq sentence-transformers 

In [71]:
!pip install ipywidgets
!pip install huggingface_hub[hf_xet]

In [72]:
!pip install tf-keras

In [73]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [74]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains.question_answering import load_qa_chain
import warnings
warnings.filterwarnings('ignore')


In [75]:
pdf = "KunalPuniaResume.pdf"
pdf_reader = PdfReader(pdf)
print(pdf_reader)

In [76]:
# extrat text from each page separately
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

print(text)

Kunal Punia
kunalpunia821@gmail.com |+91-766-807-02-94 |Leetcode |linkedin |github |GFG
Experience
Frontend Developer Intern, MV Clouds Pvt. Ltd., Ahmedabad June 2024 – July 2024
•Built multiple responsive web projects with HTML, CSS, JavaScript, and React.js, focusing on performance and
UI/UX best practices.
•Demonstrated strong work ethic, problem-solving abilities, and a keen interest in frontend design patterns.
Frontend Development Intern, Edunet Foundation June 2024 – Aug 2024
•Created and improved portfolio websites, enhancing user experience and reducing bounce rates through
well-structured layouts.
•Leveraged technologies like HTML, CSS, and JavaScript to create visually appealing and functional web pages.
Education
SRM AP UNIVERSITY , B.Tech in Computer Science and Engineering
GPA: 8.76/102022–2026
Amaravati, India
Sainik School Amaravathinagar , Higher Secondary Certificate
12th Grade Percentage: 85.22018–2022
Coimbatore, India
Projects
Stock Market Time Series Forecasting (

In [77]:
# Split the long text into small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700,
                                               chunk_overlap=200,
                                               length_function=len)

chunks = text_splitter.split_text(text=text)
chunks

['Kunal Punia\nkunalpunia821@gmail.com |+91-766-807-02-94 |Leetcode |linkedin |github |GFG\nExperience\nFrontend Developer Intern, MV Clouds Pvt. Ltd., Ahmedabad June 2024 – July 2024\n•Built multiple responsive web projects with HTML, CSS, JavaScript, and React.js, focusing on performance and\nUI/UX best practices.\n•Demonstrated strong work ethic, problem-solving abilities, and a keen interest in frontend design patterns.\nFrontend Development Intern, Edunet Foundation June 2024 – Aug 2024\n•Created and improved portfolio websites, enhancing user experience and reducing bounce rates through\nwell-structured layouts.',
 'Frontend Development Intern, Edunet Foundation June 2024 – Aug 2024\n•Created and improved portfolio websites, enhancing user experience and reducing bounce rates through\nwell-structured layouts.\n•Leveraged technologies like HTML, CSS, and JavaScript to create visually appealing and functional web pages.\nEducation\nSRM AP UNIVERSITY , B.Tech in Computer Science and

In [78]:
chunks[0]

'Kunal Punia\nkunalpunia821@gmail.com |+91-766-807-02-94 |Leetcode |linkedin |github |GFG\nExperience\nFrontend Developer Intern, MV Clouds Pvt. Ltd., Ahmedabad June 2024 – July 2024\n•Built multiple responsive web projects with HTML, CSS, JavaScript, and React.js, focusing on performance and\nUI/UX best practices.\n•Demonstrated strong work ethic, problem-solving abilities, and a keen interest in frontend design patterns.\nFrontend Development Intern, Edunet Foundation June 2024 – Aug 2024\n•Created and improved portfolio websites, enhancing user experience and reducing bounce rates through\nwell-structured layouts.'

In [79]:
chunks[1]

'Frontend Development Intern, Edunet Foundation June 2024 – Aug 2024\n•Created and improved portfolio websites, enhancing user experience and reducing bounce rates through\nwell-structured layouts.\n•Leveraged technologies like HTML, CSS, and JavaScript to create visually appealing and functional web pages.\nEducation\nSRM AP UNIVERSITY , B.Tech in Computer Science and Engineering\nGPA: 8.76/102022–2026\nAmaravati, India\nSainik School Amaravathinagar , Higher Secondary Certificate\n12th Grade Percentage: 85.22018–2022\nCoimbatore, India\nProjects\nStock Market Time Series Forecasting (Python, Streamlit, YFinance, Prophet, Plotly,\nScikit-learn )ὑ7'

In [80]:
import os

In [81]:
groq_api_key = "gsk_Vuss4ugGXF"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "jeuPBClAoThtwF"

In [82]:
llm = ChatGroq(
    model_name="llama3-8b-8192",  # or `mixtral-8x7b-32768`
)

In [83]:
response = llm.invoke("Explain me about the cricket in 100 line?")

In [84]:
print(response.content)

Cricket is a popular team sport played with a bat and ball on a grass pitch. Here's a brief overview:

**Gameplay**: Two teams of 11 players each take turns to bat and bowl. The objective is to score runs by hitting the ball and running between wickets.

**Equipment**: The game requires a cricket bat, ball, wickets (three vertical stumps and two horizontal stumps), and protective gear like helmets and pads.

**Types of deliveries**: The bowler delivers the ball in different ways, including fast balls, spin balls, and swing balls, which make it challenging for batsmen to hit.

**Shots**: Batsmen play various shots, such as drives, cuts, pulls, and hooks, to score runs. The type of shot depends on the type of delivery and the batsman's skill.

**Ways to get out**: A batsman can get out in several ways, including bowled (hit by the ball), lbw (leg before wicket), caught (hit by the ball and caught by a fielder), and stumped (hit by the ball while trying to score a run).

**Innings**: Each

In [85]:
#This working well

In [86]:
def groq_llm(groq_api_key, chunks, analyze):
    # Embeddings (you can replace with OpenAIEmbeddings if needed)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create a FAISS vector store from text chunks
    vectorstores = FAISS.from_texts(chunks, embedding=embeddings)

    # Search for similar documents
    docs = vectorstores.similarity_search(query=analyze, k=3)

    # Initialize ChatGroq with LLaMA3 model
    llm = ChatGroq(
        model_name="llama3-8b-8192"
    )

    # Load QA chain
    chain = load_qa_chain(llm=llm, chain_type='stuff')

    # Run the chain
    response = chain.run(input_documents=docs, question=analyze)

    return response

In [87]:
def resume_summary(query_with_chunks):
    query = f'''Need a detailed summarization of the below resume and a final conclusion:

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
{query_with_chunks}
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
'''
    return query

# Create the summarization query
summary = resume_summary(query_with_chunks=chunks)

# Call the Groq-based LLM function
summary_result = groq_llm(groq_api_key=groq_api_key, chunks=chunks, analyze=summary)

# Print the result
print(summary_result)


Here is a detailed summarization of the resume:

**Education**

* B.Tech in Computer Science and Engineering from SRM AP UNIVERSITY (2022-2026) with a GPA of 8.76
* Higher Secondary Certificate from Sainik School Amaravathinagar (2018-2022) with a percentage of 85.22

**Experience**

* Frontend Developer Intern at MV Clouds Pvt. Ltd., Ahmedabad (June 2024 - July 2024)
	+ Built multiple responsive web projects with HTML, CSS, JavaScript, and React.js
	+ Demonstrated strong work ethic, problem-solving abilities, and a keen interest in frontend design patterns
* Frontend Development Intern at Edunet Foundation (June 2024 - Aug 2024)
	+ Created and improved portfolio websites, enhancing user experience and reducing bounce rates
	+ Leveraged technologies like HTML, CSS, and JavaScript to create visually appealing and functional web pages

**Projects**

* Stock Market Time Series Forecasting (Python, Streamlit, YFinance, Prophet, Plotly, Scikit-learn)
	+ Developed a web app to forecast stock

In [88]:
def resume_strength(query_with_chunks):
    query = f'''need to detailed analysis and explain of the strength of below resume and finally conclude them
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                {query_with_chunks}
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

strength = resume_strength(query_with_chunks=summary_result)
strength_result = groq_llm(groq_api_key=groq_api_key, chunks=chunks, analyze=strength)
print(strength_result)

Here's a detailed analysis and explanation of the strengths of the given resume:

Strengths:

1. **Technical Skills**: Kunal Punia has a wide range of technical skills, including programming languages (Python, C++, JavaScript), web development technologies (HTML, CSS, React.js, Flask), data analysis and visualization tools (Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn, SQL, Excel, Power BI, Tableau), and machine learning and deep learning techniques (Regression, Classification, XGBoost, Random Forest, ANN, CNN, RNN, Transformers). This showcases his versatility and ability to work with various tools and technologies.

2. **Project Experience**: Kunal has worked on several projects that demonstrate his skills and accomplishments. These projects include stock market time series forecasting, customer churn prediction, and AI-powered resume analysis. Each project showcases his ability to apply his technical skills to real-world problems and deliver tangible results.

3. **Internship Ex

In [93]:
def resume_weakness(query_with_chunks):
    query = f'''
You are a professional resume reviewer. Based on the following resume content, identify **weaknesses**, **gaps**, or **areas of improvement** ONLY. 

Do NOT repeat the full summary.

Instead, list:
1. Areas that are vague, unclear, or generic.
2. Missing or weak skills for targeted roles.
3. Structural or formatting flaws.
4. Suggestions to make it more compelling or job-specific.

Resume:
---------------------------------------------------
{query_with_chunks}
---------------------------------------------------
'''
    return query

weakness = resume_weakness(query_with_chunks=summary_result)
result_weakness = groq_llm(groq_api_key=groq_api_key, chunks=chunks, analyze=weakness)
print(result_weakness)

Based on the provided resume content, the following are the identified weaknesses, gaps, or areas of improvement:

**Weaknesses:**

1. Areas that are vague, unclear, or generic:
	* The summary and conclusion sections are quite generic and do not provide specific examples or achievements.
	* The "strong work ethic, problem-solving abilities, and a keen interest in frontend design patterns" mentioned in the internship experience are too vague and could be replaced with specific examples.
2. Missing or weak skills for targeted roles:
	* The resume lacks specific skills or technologies relevant to the frontend development role, such as React Native, Angular, or Vue.js.
	* The lack of experience with version control systems like Git is notable, especially for a frontend developer.
3. Structural or formatting flaws:
	* The formatting of the skills section is unclear, making it difficult to read and understand the candidate's skills.
	* The projects section is quite lengthy and could be broke

In [94]:
def job_title_suggestion(query_with_chunks):

    query = f''' what are the job roles i apply to likedin based on below?
                  
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                  {query_with_chunks}
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

suggestion = job_title_suggestion(query_with_chunks=summary_result)
result_suggestion = groq_llm(groq_api_key=groq_api_key, chunks=chunks, analyze=suggestion)
print(result_suggestion)

Based on the provided resume, here are some job roles that Kunal Punia can apply to on LinkedIn:

1. Frontend Developer: With experience in building responsive web projects using HTML, CSS, JavaScript, and React.js, Kunal can apply for frontend developer positions.
2. Web Developer: His skills in HTML, CSS, JavaScript, and React.js make him a strong candidate for web developer roles.
3. UI/UX Designer: With a keen interest in frontend design patterns, Kunal can apply for UI/UX designer positions.
4. Data Analyst: His skills in data analysis and visualization using Pandas, NumPy, Scikit-learn, Matplotlib, and Seaborn make him a strong candidate for data analyst roles.
5. Machine Learning Engineer: His experience in machine learning and deep learning using XGBoost, Random Forest, ANN, CNN, RNN, and Transformers makes him a strong candidate for machine learning engineer positions.
6. AI/ML Developer: With his skills in AI and machine learning using LangChain, OpenAI API, Hugging Face, FAI